In [1]:
import pandas as pd
import numpy as np
import nltk
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
from nltk.tokenize import word_tokenize
!pip install wordcloud
from nltk.corpus import stopwords
nltk.download('stopwords')
stop=stopwords.words("english")
nltk.download('punkt')

from sklearn.ensemble import StackingClassifier

  ERROR: Command errored out with exit status 1:
   command: 'C:\Users\perik\anaconda3\python.exe' -u -c 'import io, os, sys, setuptools, tokenize; sys.argv[0] = '"'"'C:\\Users\\perik\\AppData\\Local\\Temp\\pip-install-xjyl82cd\\wordcloud_48bff27a24924808b885ddfae44f085a\\setup.py'"'"'; __file__='"'"'C:\\Users\\perik\\AppData\\Local\\Temp\\pip-install-xjyl82cd\\wordcloud_48bff27a24924808b885ddfae44f085a\\setup.py'"'"';f = getattr(tokenize, '"'"'open'"'"', open)(__file__) if os.path.exists(__file__) else io.StringIO('"'"'from setuptools import setup; setup()'"'"');code = f.read().replace('"'"'\r\n'"'"', '"'"'\n'"'"');f.close();exec(compile(code, __file__, '"'"'exec'"'"'))' bdist_wheel -d 'C:\Users\perik\AppData\Local\Temp\pip-wheel-abotnfag'
       cwd: C:\Users\perik\AppData\Local\Temp\pip-install-xjyl82cd\wordcloud_48bff27a24924808b885ddfae44f085a\
  Complete output (20 lines):
  running bdist_wheel
  running build
  running build_py
  creating build
  creating build\lib.win-amd64-3.9

  Using cached wordcloud-1.8.1.tar.gz (220 kB)
  Running setup.py clean for wordcloud
Failed to build wordcloud
    Running setup.py install for wordcloud: started
    Running setup.py install for wordcloud: finished with status 'error'


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\perik\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\perik\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
data=pd.read_csv(r"/content/spam_data.csv")
data.shape

FileNotFoundError: [Errno 2] No such file or directory: '/content/spam_data.csv'

In [ ]:
data.groupby('Category').describe().T

In [ ]:
data.isnull()

In [ ]:
data=data.dropna()
data

In [ ]:
data["Category"].value_counts().plot(kind = 'pie', explode = [0, 0.1], figsize = (6, 6))
plt.ylabel("Spam vs Ham")
plt.legend(["Ham", "Spam"])
plt.show()

In [ ]:
print(data.dtypes)
data['length']=data.Message.apply(len)
data.head()

In [ ]:
data.hist(column='length', by='Category', bins=50,figsize=(11,5))

In [ ]:
spam_msgs=data[data['Category']=='spam'].Message
ham_msgs=data[data['Category']=='ham'].Message

In [ ]:
spam_words = []
ham_words = []

def extractSpamWords(spamMessages):
    global spam_words
    words = [word for word in word_tokenize(spamMessages)]
    spam_words = spam_words + words
    
def extractHamWords(hamMessages):
    global ham_words
    words = [word for word in word_tokenize(hamMessages) ]
    ham_words = ham_words + words

spam_msgs.apply(extractSpamWords)
ham_msgs.apply(extractHamWords)

In [ ]:
ham_words

In [ ]:
!pip install wordcloud
from wordcloud import WordCloud

In [ ]:
spam_wordcloud = WordCloud(width=600, height=400).generate(" ".join(spam_words))
plt.figure( figsize=(10,8), facecolor='k')
plt.imshow(spam_wordcloud)

In [ ]:
from wordcloud import WordCloud
ham_wordcloud = WordCloud(width=600, height=400).generate(" ".join(ham_words))
plt.figure( figsize=(10,8))
plt.imshow(ham_wordcloud)

In [ ]:
import string
def text_preprocessing(text):
    text=text.translate(str.maketrans('','',string.punctuation))
    text=[i.lower() for i in text.split() if i.lower() not in stop]
    return " ".join(text)
data['Message']=data['Message'].apply(text_preprocessing)

In [ ]:
def encodeCategory(cat):
    if cat == "spam":
        return 1
    else:
        return 0
        
data["Category"] = data["Category"].apply(encodeCategory)

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer()
X=cv.fit_transform(data["Message"])
print (X.shape)

In [ ]:
X = cv.fit_transform(data["Message"]).toarray()
Y=data['Category']

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size = 0.20)

In [ ]:
from sklearn.linear_model import LogisticRegression

logistic_reg=LogisticRegression(solver='liblinear',penalty='l1')
logistic_reg.fit(X_train,y_train)
lrprediction=logistic_reg.predict(X_test)

In [ ]:
from sklearn.metrics import confusion_matrix,classification_report,accuracy_score
import seaborn as sns

# performance of logisctic regression
print("Performance of logistic regression:")
print()
print('classification report:\n',classification_report(y_test,lrprediction))
print()
print("confuion matrix:\n",confusion_matrix(y_test,lrprediction))
print()
print("Accuracy score:",accuracy_score(y_test,lrprediction))
print("\n\nHeatmap on prediction:\n")
sns.heatmap(confusion_matrix(y_test,lrprediction),annot=True)

In [ ]:
from sklearn.naive_bayes import MultinomialNB

nbclassifier=MultinomialNB()
nbclassifier.fit(X_train,y_train)
nbprediction=nbclassifier.predict(X_test)

In [ ]:
# performance of Naive bayes 
print("Performance of Naive bayes:")
print()
print('classification report:\n',classification_report(y_test,nbprediction))
print()
print("confuion matrix:\n",confusion_matrix(y_test,nbprediction))
print()
print("Accuracy score:",accuracy_score(y_test,nbprediction))
print("\n\nHeatmap on prediction:\n")
sns.heatmap(confusion_matrix(y_test,nbprediction),annot=True)

In [ ]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier()
rf_model = rf.fit(X_train,y_train)
y_pred_rf=rf_model.predict(X_test)

In [ ]:
from sklearn.metrics import confusion_matrix,classification_report,accuracy_score,precision_score
import seaborn as sns

# performance of random forest
print("Performance of random forest:")
print()
print('classification report:\n',classification_report(y_test,y_pred_rf))
print()
print("confuion matrix:\n",confusion_matrix(y_test,y_pred_rf))
print()
print("Accuracy score:",accuracy_score(y_test,y_pred_rf))
print('precision score:',precision_score(y_test,y_pred_rf))
print("\n\nHeatmap on prediction:\n")
sns.heatmap(confusion_matrix(y_test,y_pred_rf),annot=True)

In [ ]:
final_estimator=RandomForestClassifier()

In [ ]:
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import AdaBoostClassifier

In [ ]:
abc = AdaBoostClassifier(n_estimators=50, random_state=2)
etc = ExtraTreesClassifier(n_estimators=50, random_state=2)

In [ ]:
estimators=[('abc',abc ), ('etc', etc)]
clf = StackingClassifier(estimators=estimators, final_estimator=final_estimator)
clf.fit(X_train,y_train)
y_pred = clf.predict(X_test)
# performance of stacked classifier
print("Performance of stacked classifier:")
print()
print('classification report:\n',classification_report(y_test,y_pred))
print()
print("confuion matrix:\n",confusion_matrix(y_test,y_pred))
print()
print("Accuracy score:",accuracy_score(y_test,y_pred))
print('precision score:',precision_score(y_test,y_pred))
print("\n\nHeatmap on prediction:\n")
sns.heatmap(confusion_matrix(y_test,y_pred),annot=True)

In [ ]:
def check(text):
    lab = ['not spam','spam'] 
    x = cv.transform([text]).toarray()
    p = clf.predict(x)
    s = [str(i) for i in p]
    a = int("".join(s))
    res = str("This message is looking: "+ lab[a])
    print(res)



In [ ]:
check('Congratulations, your entry into our contest last month made you a WINNER! goto our website to claim your price! You have 24 hours to claim.')

In [ ]:
check("I cannot attend the school for 10 days as I am going out on a vaccation kindly excuse me. I will keep my notes up to date as soon I return. Thanking you")

In [ ]:
import joblib
joblib.dump(cv,'vectorizer.sav')


In [ ]:
joblib.dump(clf,'model.sav')